# Deploying Machine Learning Models using kubectl
This demo shows how you can interact directly with kubernetes using kubectl to create and manage runtime machine learning models.
<img src="images/deploy-graph.png" alt="predictor with canary" title="ml graph"/>

In [1]:
!kubectl label nodes apiserver-crd role=locust --overwrite

node "apiserver-crd" not labeled


## Start seldon-core

In [10]:
!helm install ../../helm-charts/seldon-core --name seldon-core --set grafana_prom_admin_password=password --set persistence.enabled=false --set cluster_manager.image.tag=0.2.16_kubeapi --set apife.image.tag=0.0.7_kubeapi

NAME:   seldon-core
LAST DEPLOYED: Sat Nov  4 09:30:53 2017
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/ConfigMap
NAME                       DATA  AGE
alertmanager-server-conf   1     4s
grafana-import-dashboards  5     4s
prometheus-rules           4     4s
prometheus-server-conf     1     4s

==> v1beta1/CustomResourceDefinition
NAME                                     AGE
mldeployments.machinelearning.seldon.io  4s

==> v1/Service
NAME                      TYPE       CLUSTER-IP  EXTERNAL-IP  PORT(S)                     AGE
alertmanager              ClusterIP  10.0.0.141  <none>       80/TCP                      4s
seldon-apiserver          NodePort   10.0.0.70   <none>       8080:30032/TCP              4s
seldon-cluster-manager    NodePort   10.0.0.128  <none>       8080:30030/TCP              3s
grafana-prom              NodePort   10.0.0.169  <none>       80:30034/TCP                3s
kafka                     NodePort   10.0.0.2    <none>       9092:30010/TCP         

In [11]:
!kubectl get deployments

NAME                      DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
alertmanager-deployment   1         1         1            1           1m
grafana-prom-deployment   1         1         1            1           1m
kafka                     1         1         1            1           1m
prometheus-deployment     1         1         1            1           1m
redis                     1         1         1            1           1m
seldon-apiserver          1         1         1            1           1m
seldon-cluster-manager    1         1         1            1           1m


# Integrating with Kubernetes API

In [32]:
!cat resources/model.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha1",
    "kind": "MLDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "mldeployment-example"
    },
    "spec": {
        "annotations": {
            "project_name": "FX Market Prediction"
        },
        "cluster": {},
        "id": "1",
        "name": "test deployment",
        "oauth_key": "oauth-key",
        "oauth_secret": "oauth-secret",
        "predictor": {
            "annotations": {
                "project_name": "FX Market Prediction"
            },
            "enabled": true,
            "id": "0",
            "imagePullSecrets": [],
            "name": "FX Market Predictor",
            "predictiveUnits": [
                {
                    "children": [],
                    "cluster_resources": {
                        "cpu": "",
                        "gpu": "",
                        "id": "2",
                        "image": "seldonio/mean_cla

## Create ML Deployment

In [13]:
!kubectl apply -f resources/model.json

mldeployment "mldeployment-example" created


In [34]:
!kubectl get mldeployments

NAME                   AGE
mldeployment-example   4s


In [35]:
!kubectl describe mldeployments mldeployment-example 

Name:         mldeployment-example
Namespace:    default
Labels:       app=seldon
Annotations:  kubectl.kubernetes.io/last-applied-configuration={"apiVersion":"machinelearning.seldon.io/v1alpha1","kind":"MLDeployment","metadata":{"name":"mldeployment-example","namespace":"default","selfLink":"/a...
API Version:  machinelearning.seldon.io/v1alpha1
Kind:         MLDeployment
Metadata:
  Cluster Name:                   
  Creation Timestamp:             2017-11-01T16:22:13Z
  Deletion Grace Period Seconds:  0
  Finalizers:
  Generate Name:  
  Generation:     0
  Initializers:   <nil>
  Owner References:
  Resource Version:  22140
  Self Link:         /apis/machinelearning.seldon.io/v1alpha1/namespaces/default/mldeployments/mldeployment-example
  UID:               d1ae0d16-bf20-11e7-ba47-08002751d93c
Spec:
  Annotations:
    Project _ Name:  FX Market Prediction
  Cluster:
  Id:              1
  Name:            test deployment
  Oauth _ Key:     oauth-key
  Oauth _ Secret:  oauth-secret

In [36]:
!kubectl get rs --selector=app=sd-1-p

NAME               DESIRED   CURRENT   READY     AGE
sd-1-p-c78c8f77f   1         1         1         43s


## Get predictions

In [37]:
%%bash
API_SERVER=35.195.94.148:8080
MINIKUBE_SERVER=192.168.99.100:30032
SERVER=$API_SERVER
SERVER=$MINIKUBE_SERVER
#SERVER=0.0.0.0:8080
echo "$SERVER"
TOKEN=`curl -s -H "Accept: application/json" oauth-key:oauth-secret@${SERVER}/oauth/token -d grant_type=client_credentials | jq -r '.access_token'`
curl -s -H "Content-Type:application/json" -H "Accept: application/json" -H "Authorization: Bearer $TOKEN" \
    ${SERVER}/api/v0.1/predictions \
    -d '{"request":{"features":["a"],"ndarray":[[1.0],[2.0]]}}'

192.168.99.100:30032
{
  "meta": {
    "puid": "8qrh785m3gkjvol2ro96jbtpki",
    "routing": {
    }
  },
  "response": {
    "features": ["proba"],
    "ndarray": [[0.12823373759251927], [0.2856375705089441]]
  }
}

## Update deployment with canary

In [38]:
!cat resources/model_with_canary.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha1",
    "kind": "MLDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "mldeployment-example"
    },
    "spec": {
        "annotations": {
            "project_name": "FX Market Prediction"
        },
        "cluster": {},
        "id": "1",
        "name": "test deployment",
        "oauth_key": "oauth-key",
        "oauth_secret": "oauth-secret",
        "predictor": {
            "annotations": {
                "project_name": "FX Market Prediction"
            },
            "enabled": true,
            "id": "0",
            "imagePullSecrets": [],
            "name": "FX Market Predictor",
            "predictiveUnits": [
                {
                    "children": [],
                    "cluster_resources": {
                        "cpu": "",
                        "gpu": "",
                        "id": "2",
                        "image": "seldonio/mean_cla

In [39]:
!kubectl apply -f resources/model_with_canary.json

mldeployment "mldeployment-example" configured


In [47]:
!kubectl get rs --selector=app=sd-1-p

NAME                DESIRED   CURRENT   READY     AGE
sd-1-c-54f5db79cd   1         1         1         1m
sd-1-p-c78c8f77f    1         1         1         2m


In [50]:
%%bash
API_SERVER=35.189.249.200:8080
MINIKUBE_SERVER=192.168.99.100:30032
SERVER=$API_SERVER
SERVER=$MINIKUBE_SERVER
TOKEN=`curl -s -H "Accept: application/json" oauth-key:oauth-secret@${SERVER}/oauth/token -d grant_type=client_credentials | jq -r '.access_token'`
curl -s -H "Content-Type:application/json" -H "Accept: application/json" -H "Authorization: Bearer $TOKEN" \
    ${SERVER}/api/v0.1/predictions \
    -d '{"request":{"features":["a"],"ndarray":[[1.0],[2.0]]}}'

{
  "meta": {
    "puid": "qumjp3j0a9ufq65r0bak2ggh3j",
    "routing": {
    }
  },
  "response": {
    "features": ["proba"],
    "ndarray": [[0.12823373759251927], [0.2856375705089441]]
  }
}

## Load test

In [48]:
!helm install --set oauth.key=oauth-key --set oauth.secret=oauth-secret --name loadtest ../../helm-charts/seldon-core-loadtesting

NAME:   loadtest
LAST DEPLOYED: Wed Nov  1 16:26:50 2017
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/Service
NAME             TYPE      CLUSTER-IP  EXTERNAL-IP  PORT(S)                                       AGE
locust-master-1  NodePort  10.0.0.122  <none>       5557:31745/TCP,5558:32324/TCP,8089:31532/TCP  1s

==> v1/Pod(related)
NAME                   READY  STATUS             RESTARTS  AGE
locust-slave-1-rrg26   0/1    ContainerCreating  0         1s
locust-master-1-fgbdx  0/1    ContainerCreating  0         1s

==> v1/ReplicationController
NAME             DESIRED  CURRENT  READY  AGE
locust-slave-1   1        1        0      1s
locust-master-1  1        1        0      1s




# Tear down

In [8]:
!kubectl delete -f resources/model_with_canary.json

mldeployment "mldeployment-example" deleted


In [52]:
!helm delete loadtest --purge

release "loadtest" deleted


In [9]:
!helm delete seldon-core --purge

release "seldon-core" deleted
